In [1]:
# READ EXCEL - TRANSFORM TO CSV
import pandas as pd 
import os

file_path = '../data/matriculas_ed_superior_nuble_2021.xlsx'
df = pd.read_excel(file_path)

# check if files exists, if yes, don't overwrite
if os.path.exists('../data/matriculas_ed_superior_nuble_2021.csv'):
    print('File already exists, exiting without overwriting.')
else:
    df.to_csv('../data/matriculas_ed_superior_nuble_2021.csv', index=False)

File already exists, exiting without overwriting.


In [2]:
# HELPER FUNCTIONS
def get_mapping(data_column):
    mapping = {}
    column_types = df[data_column].dropna().unique()
    for i, tipo in enumerate(sorted(column_types)):
        mapping[tipo] = i + 1
    return mapping

In [4]:
# COMPREHENSIVE MAPPINGS FOR MATRICULAS TABLE

# 1. Simple mappings
genero_map = get_mapping('GENERO')
rango_edad_map = get_mapping('RANGO EDAD')
via_ingreso_map = get_mapping('VIA DE INGRESO')

print("=== SIMPLE MAPPINGS ===")
print("Genero mapping:", genero_map)
print("Rango edad mapping:", rango_edad_map)
print("Via ingreso mapping:", via_ingreso_map)

# 2. Complex mapping for INSTITUCION
def get_institucion_mapping():
    """
    Create mapping for institutions based on:
    - TIPO DE INSTITUCION
    - NOMBRE DE INSTITUCION  
    - ACREDITACION INSTITUCIONAL
    - PERIODO DE ACREDITACION
    """
    mapping = {}
    
    # Get unique combinations
    unique_combinations = set()
    for _, row in df.iterrows():
        tipo = row['TIPO DE INSTITUCION']
        nombre = row['NOMBRE DE INSTITUCION']
        acreditacion = row['ACREDITACION INSTITUCIONAL']
        periodo = row['PERIODO DE ACREDITACION']
        
        if pd.notna(tipo) and pd.notna(nombre) and pd.notna(acreditacion):
            # Handle potential NaN in periodo
            periodo_str = str(periodo) if pd.notna(periodo) else 'NULL'
            combination = (tipo, nombre, acreditacion, periodo_str)
            unique_combinations.add(combination)
    
    # Assign sequential IDs
    for i, combination in enumerate(sorted(unique_combinations)):
        mapping[combination] = i + 1
    
    return mapping

institucion_map = get_institucion_mapping()
print(f"\n=== INSTITUCION MAPPING ===")
print(f"Found {len(institucion_map)} unique institutions")
print("First 5 institutions:")
for i, (combo, id_val) in enumerate(list(institucion_map.items())[:5]):
    print(f"  {id_val}: {combo}")

# 3. Complex mapping for CARRERA (using all specified columns)
def get_carrera_mapping():
    """
    Create mapping for careers based on all career-related columns
    """
    mapping = {}
    
    # Get unique combinations
    unique_combinations = set()
    for _, row in df.iterrows():
        nombre_carrera = row['NOMBRE CARRERA']
        requisito_ingreso = row['REQUISITO INGRESO']
        modalidad = row['MODALIDAD']
        jornada = row['JORNADA']
        tipo_plan_carrera = row['TIPO PLAN CARRERA']
        nivel_estudio_carrera = row['NIVEL DE ESTUDIO CARRERA']
        nivel_carrera = row['NIVEL CARRERA']
        area_conocimiento = row['AREA CONOCIMIENTO']
        duracion_plan = row['DURACION PLAN DE ESTUDIO (SEMESTRES)']
        duracion_titulacion = row['DURACION PROCESO TITULACION (SEMESTRES)']
        duracion_total = row['DURACION TOTAL CARRERA (SEMESTRES)']
        valor_matricula = row['VALOR MATRICULA (PESOS)']
        valor_arancel = row['VALOR ARANCEL (PESOS)']
        
        # Check if all required fields are not null
        if all(pd.notna(val) for val in [nombre_carrera, requisito_ingreso, modalidad, jornada, 
                                        tipo_plan_carrera, nivel_estudio_carrera, nivel_carrera, 
                                        area_conocimiento, duracion_plan, duracion_titulacion, 
                                        duracion_total, valor_matricula, valor_arancel]):
            
            combination = (nombre_carrera, requisito_ingreso, modalidad, jornada, tipo_plan_carrera,
                          nivel_estudio_carrera, nivel_carrera, area_conocimiento, 
                          int(duracion_plan), int(duracion_titulacion), int(duracion_total),
                          int(valor_matricula), int(valor_arancel))
            unique_combinations.add(combination)
    
    # Assign sequential IDs
    for i, combination in enumerate(sorted(unique_combinations)):
        mapping[combination] = i + 1
    
    return mapping

carrera_map = get_carrera_mapping()
print(f"\n=== CARRERA MAPPING ===")
print(f"Found {len(carrera_map)} unique careers")
print("First 3 careers (showing first 5 fields only):")
for i, (combo, id_val) in enumerate(list(carrera_map.items())[:3]):
    print(f"  {id_val}: {combo[:5]}...")  # Show only first 5 fields for readability

=== SIMPLE MAPPINGS ===
Genero mapping: {'Femenino': 1, 'Masculino': 2}
Rango edad mapping: {'15 a 19 ': 1, '20 a 24 ': 2, '25 a 29 ': 3, '30 a 34 ': 4, '35 a 39 ': 5, '40 y mas': 6}
Via ingreso mapping: {'Articulacion de TNS a carrera profesional': 1, 'Cambio Interno': 2, 'Cambio externo': 3, 'Continuidad desde plan comun o Bachillerato.': 4, 'Ingreso Directo (regular)': 5, 'Ingreso a traves de PACE': 6, 'Ingreso a traves de programas de inclusion': 7, 'Ingreso especial para estudiantes extranjeros': 8, 'Ingreso por Reconocimiento de Aprendizajes Previos (RAP)': 9, 'Ingreso por caracteristicas especiales': 10, 'Otras formas de Ingreso': 11}

=== INSTITUCION MAPPING ===
Found 15 unique institutions
First 5 institutions:
  1: ('Centros de Formacion Tecnica', 'CFT INACAP', 'ACREDITADA', '05/01/2018 AL 05/01/2025')
  2: ('Centros de Formacion Tecnica', 'CFT SANTO TOMAS', 'ACREDITADA', '20/12/2019 AL 20/12/2024')
  3: ('Institutos Profesionales', 'IP DEL VALLE CENTRAL', 'NO ACREDITADA', 'N

In [5]:
# CREATE MATRICULAS DATAFRAME FOR DATABASE
def map_institucion_id(row):
    """Map a row to its institution ID"""
    tipo = row['TIPO DE INSTITUCION']
    nombre = row['NOMBRE DE INSTITUCION']
    acreditacion = row['ACREDITACION INSTITUCIONAL']
    periodo = str(row['PERIODO DE ACREDITACION']) if pd.notna(row['PERIODO DE ACREDITACION']) else 'NULL'
    
    combination = (tipo, nombre, acreditacion, periodo)
    return institucion_map.get(combination, None)

def map_carrera_id(row):
    """Map a row to its career ID"""
    nombre_carrera = row['NOMBRE CARRERA']
    requisito_ingreso = row['REQUISITO INGRESO']
    modalidad = row['MODALIDAD']
    jornada = row['JORNADA']
    tipo_plan_carrera = row['TIPO PLAN CARRERA']
    nivel_estudio_carrera = row['NIVEL DE ESTUDIO CARRERA']
    nivel_carrera = row['NIVEL CARRERA']
    area_conocimiento = row['AREA CONOCIMIENTO']
    duracion_plan = row['DURACION PLAN DE ESTUDIO (SEMESTRES)']
    duracion_titulacion = row['DURACION PROCESO TITULACION (SEMESTRES)']
    duracion_total = row['DURACION TOTAL CARRERA (SEMESTRES)']
    valor_matricula = row['VALOR MATRICULA (PESOS)']
    valor_arancel = row['VALOR ARANCEL (PESOS)']
    
    # Check if all required fields are not null
    if all(pd.notna(val) for val in [nombre_carrera, requisito_ingreso, modalidad, jornada, 
                                    tipo_plan_carrera, nivel_estudio_carrera, nivel_carrera, 
                                    area_conocimiento, duracion_plan, duracion_titulacion, 
                                    duracion_total, valor_matricula, valor_arancel]):
        
        combination = (nombre_carrera, requisito_ingreso, modalidad, jornada, tipo_plan_carrera,
                      nivel_estudio_carrera, nivel_carrera, area_conocimiento, 
                      int(duracion_plan), int(duracion_titulacion), int(duracion_total),
                      int(valor_matricula), int(valor_arancel))
        return carrera_map.get(combination, None)
    return None

# Create matriculas DataFrame
matriculas_df = df.copy()

# Apply mappings
matriculas_df['genero_id'] = matriculas_df['GENERO'].map(genero_map)
matriculas_df['rango_edad_id'] = matriculas_df['RANGO EDAD'].map(rango_edad_map)
matriculas_df['via_ingreso_id'] = matriculas_df['VIA DE INGRESO'].map(via_ingreso_map)
matriculas_df['institucion_id'] = matriculas_df.apply(map_institucion_id, axis=1)
matriculas_df['carrera_id'] = matriculas_df.apply(map_carrera_id, axis=1)

# Create final matriculas DataFrame for database
matriculas_for_db = matriculas_df[[
    'ID', 'genero_id', 'EDAD', 'rango_edad_id', 'AÑO INGRESO', 'SEMESTRE INGRESO',
    'institucion_id', 'carrera_id', 'via_ingreso_id'
]].copy()

# Rename columns to match database schema
matriculas_for_db = matriculas_for_db.rename(columns={
    'ID': 'matricula_id',
    'EDAD': 'edad',
    'AÑO INGRESO': 'anio_ingreso',
    'SEMESTRE INGRESO': 'semestre_ingreso'
})

# Add comuna_id as constant 147 as specified
matriculas_for_db['comuna_id'] = 147

# Check for any unmapped values
print("=== MATRICULAS MAPPING VALIDATION ===")
print("Unmapped generos:", matriculas_for_db[matriculas_for_db['genero_id'].isna()].shape[0])
print("Unmapped rango edad:", matriculas_for_db[matriculas_for_db['rango_edad_id'].isna()].shape[0])
print("Unmapped vias ingreso:", matriculas_for_db[matriculas_for_db['via_ingreso_id'].isna()].shape[0])
print("Unmapped instituciones:", matriculas_for_db[matriculas_for_db['institucion_id'].isna()].shape[0])
print("Unmapped carreras:", matriculas_for_db[matriculas_for_db['carrera_id'].isna()].shape[0])

print(f"\nFinal matriculas DataFrame shape: {matriculas_for_db.shape}")
print("Columns:", list(matriculas_for_db.columns))
print("\nFirst 5 rows:")
print(matriculas_for_db.head())

print(f"\nSemestre ingreso values: {matriculas_for_db['semestre_ingreso'].unique()}")
print(f"Years range: {matriculas_for_db['anio_ingreso'].min()} - {matriculas_for_db['anio_ingreso'].max()}")
print(f"Age range: {matriculas_for_db['edad'].min()} - {matriculas_for_db['edad'].max()}")

=== MATRICULAS MAPPING VALIDATION ===
Unmapped generos: 0
Unmapped rango edad: 0
Unmapped vias ingreso: 0
Unmapped instituciones: 0
Unmapped carreras: 0

Final matriculas DataFrame shape: (18785, 10)
Columns: ['matricula_id', 'genero_id', 'edad', 'rango_edad_id', 'anio_ingreso', 'semestre_ingreso', 'institucion_id', 'carrera_id', 'via_ingreso_id', 'comuna_id']

First 5 rows:
   matricula_id  genero_id  edad  rango_edad_id  anio_ingreso  \
0            16          1    21              2          2019   
1            21          2    20              2          2021   
2            22          2    20              2          2021   
3           175          2    19              1          2020   
4           222          2    32              4          2020   

  semestre_ingreso  institucion_id  carrera_id  via_ingreso_id  comuna_id  
0  Primer semestre               1           1               5        147  
1  Primer semestre               1         310               5        147  
2  

In [8]:
# SAVE MATRICULAS DATA TO CSV AND CREATE SQL INSERT SCRIPT

# Save to CSV
os.makedirs('../seeds', exist_ok=True)
matriculas_for_db.to_csv('../seeds/matriculas.csv', index=False, na_rep='NULL')

def create_matriculas_sql_script(df, filename='../seeds/matriculas_inserts.sql'):
    """
    Creates an Oracle SQL script with INSERT statements for the matriculas table
    """
    sql_lines = []
    sql_lines.append("-- INSERT statements for MATRICULAS table")
    sql_lines.append("-- Generated automatically from ETL process")
    sql_lines.append(f"-- Total records: {len(df)}")
    sql_lines.append("")
    
    # Process in batches to avoid memory issues with large datasets
    batch_size = 1000
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]
        
        sql_lines.append(f"-- Batch {batch_num + 1} of {total_batches} (rows {start_idx + 1} to {end_idx})")
        
        for index, row in batch_df.iterrows():
            # Format values for Oracle SQL
            matricula_id = int(row['matricula_id'])
            genero_id = int(row['genero_id'])
            edad = int(row['edad'])
            rango_edad_id = int(row['rango_edad_id'])
            anio_ingreso = int(row['anio_ingreso'])
            semestre_ingreso = int(1 if row['semestre_ingreso'] == 'Primer semestre' else 2)
            institucion_id = int(row['institucion_id'])
            carrera_id = int(row['carrera_id'])
            via_ingreso_id = int(row['via_ingreso_id'])
            comuna_id = int(row['comuna_id'])
            
            # Create INSERT statement
            insert_stmt = f"""INSERT INTO matriculas (matricula_id, genero_id, edad, rango_edad_id, anio_ingreso, semestre_ingreso, institucion_id, carrera_id, via_ingreso_id, comuna_id) 
VALUES ({matricula_id}, {genero_id}, {edad}, {rango_edad_id}, {anio_ingreso}, {semestre_ingreso}, {institucion_id}, {carrera_id}, {via_ingreso_id}, {comuna_id});"""
            
            sql_lines.append(insert_stmt)
        
        sql_lines.append("")
        if batch_num < total_batches - 1:  # Don't commit after last batch yet
            sql_lines.append("COMMIT;")
            sql_lines.append("")
    
    sql_lines.append("COMMIT;")
    sql_lines.append("-- End of matriculas inserts")
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    return len(df)

# Create the SQL script
num_inserts = create_matriculas_sql_script(matriculas_for_db)
print(f"✅ Created matriculas.csv with {len(matriculas_for_db)} records")
print(f"✅ Created matriculas_inserts.sql with {num_inserts} INSERT statements")
print("Files saved to:")
print("  - ../seeds/matriculas.csv")
print("  - ../seeds/matriculas_inserts.sql")

# Show sample of the generated SQL
with open('../seeds/matriculas_inserts.sql', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print(f"\nFirst 10 lines of generated SQL (from {len(lines)} total lines):")
    for i, line in enumerate(lines[:10]):
        print(f"{i+1:2d}: {line.rstrip()}")

print("\n=== SUMMARY ===")
print(f"Total matriculas records: {len(matriculas_for_db)}")
print(f"Unique institutions: {len(institucion_map)}")
print(f"Unique careers: {len(carrera_map)}")
print(f"Unique genders: {len(genero_map)}")
print(f"Unique age ranges: {len(rango_edad_map)}")
print(f"Unique entrance methods: {len(via_ingreso_map)}")
print(f"Comuna ID (constant): 147")
print("\n✅ All matriculas data successfully processed and ready for database insertion!")

✅ Created matriculas.csv with 18785 records
✅ Created matriculas_inserts.sql with 18785 INSERT statements
Files saved to:
  - ../seeds/matriculas.csv
  - ../seeds/matriculas_inserts.sql

First 10 lines of generated SQL (from 37650 total lines):
 1: -- INSERT statements for MATRICULAS table
 2: -- Generated automatically from ETL process
 3: -- Total records: 18785
 4: 
 5: -- Batch 1 of 19 (rows 1 to 1000)
 6: INSERT INTO matriculas (matricula_id, genero_id, edad, rango_edad_id, anio_ingreso, semestre_ingreso, institucion_id, carrera_id, via_ingreso_id, comuna_id)
 7: VALUES (16, 1, 21, 2, 2019, 1, 1, 1, 5, 147);
 8: INSERT INTO matriculas (matricula_id, genero_id, edad, rango_edad_id, anio_ingreso, semestre_ingreso, institucion_id, carrera_id, via_ingreso_id, comuna_id)
 9: VALUES (21, 2, 20, 2, 2021, 1, 1, 310, 5, 147);
10: INSERT INTO matriculas (matricula_id, genero_id, edad, rango_edad_id, anio_ingreso, semestre_ingreso, institucion_id, carrera_id, via_ingreso_id, comuna_id)

=== 

FROM THIS:
INSERT INTO matriculas (matricula_id, genero_id, edad, rango_edad_id, anio_ingreso, semestre_ingreso, institucion_id, carrera_id, via_ingreso_id, comuna_id) 
VALUES (16, 1, 21, 2, 2019, 1, 1, 1, 5, 147);

TO THIS:
-- Record 1
IF execute_matricula_insert(16, 1, 21, 2, 2019, 1, 1, 1, 5, 147) THEN
    v_processed_count := v_processed_count + 1;
ELSE
    v_processed_count := v_processed_count + 1;
END IF;

IF MOD(v_processed_count, 100) = 0 THEN
    process_matricula_batch(v_batch_commit_size, v_max_errors, v_success_count, v_error_count, v_should_abort);
    IF v_should_abort THEN
        RAISE_APPLICATION_ERROR(-20002, 'Process aborted due to excessive errors (' || v_error_count || ')');
    END IF;
END IF;

In [12]:
# CREATE SQL INSERT SCRIPT USING A FUNCTION

def create_matriculas_sql_script(df, filename='../seeds/matriculas_functions_seed.sql'):
    """
    Creates an Oracle SQL script with INSERT statements for the matriculas table
    """
    sql_lines = []
    sql_lines.append("-- FUNCTION calls for MATRICULAS table")
    sql_lines.append("-- Generated automatically from ETL process")
    sql_lines.append(f"-- Total records: {len(df)}")
    sql_lines.append("")
    
    # Process in batches to avoid memory issues with large datasets
    batch_size = 1000
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]
        
        sql_lines.append(f"-- Batch {batch_num + 1} of {total_batches} (rows {start_idx + 1} to {end_idx})")
        
        for index, row in batch_df.iterrows():
            # Format values for Oracle SQL
            matricula_id = int(row['matricula_id'])
            genero_id = int(row['genero_id'])
            edad = int(row['edad'])
            rango_edad_id = int(row['rango_edad_id'])
            anio_ingreso = int(row['anio_ingreso'])
            semestre_ingreso = int(1 if row['semestre_ingreso'] == 'Primer semestre' else 2)
            institucion_id = int(row['institucion_id'])
            carrera_id = int(row['carrera_id'])
            via_ingreso_id = int(row['via_ingreso_id'])
            comuna_id = int(row['comuna_id'])
            
            # Create INSERT statement
            insert_stmt = f"""execute_matricula_insert({matricula_id}, {genero_id}, {edad}, {rango_edad_id}, {anio_ingreso}, {semestre_ingreso}, {institucion_id}, {carrera_id}, {via_ingreso_id}, {comuna_id});"""
            
            sql_lines.append(insert_stmt)
        
        sql_lines.append("")
        if batch_num < total_batches - 1:  # Don't commit after last batch yet
            sql_lines.append("")
    
    sql_lines.append("-- End of matriculas inserts")
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    return len(df)

# Create the SQL script
num_inserts = create_matriculas_sql_script(matriculas_for_db)
print(f"✅ Created matriculas_inserts.sql with {num_inserts} Functions calls")
print("File saved to:")
print("  - ../seeds/matriculas_functions_seed.sql")

# Show sample of the generated SQL
with open('../seeds/matriculas_functions_seed.sql', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print(f"\nFirst 10 lines of generated SQL (from {len(lines)} total lines):")
    for i, line in enumerate(lines[:10]):
        print(f"{i+1:2d}: {line.rstrip()}")

print("\n=== SUMMARY ===")
print(f"Total matriculas records: {len(matriculas_for_db)}")
print(f"Unique institutions: {len(institucion_map)}")
print(f"Unique careers: {len(carrera_map)}")
print(f"Unique genders: {len(genero_map)}")
print(f"Unique age ranges: {len(rango_edad_map)}")
print(f"Unique entrance methods: {len(via_ingreso_map)}")
print(f"Comuna ID (constant): 147")
print("\n✅ All matriculas data successfully processed and ready for database insertion!")

✅ Created matriculas_inserts.sql with 18785 Functions calls
File saved to:
  - ../seeds/matriculas_functions_seed.sql

First 10 lines of generated SQL (from 18846 total lines):
 1: -- FUNCTION calls for MATRICULAS table
 2: -- Generated automatically from ETL process
 3: -- Total records: 18785
 4: 
 5: -- Batch 1 of 19 (rows 1 to 1000)
 6: execute_matricula_insert(16, 1, 21, 2, 2019, 1, 1, 1, 5, 147);
 7: execute_matricula_insert(21, 2, 20, 2, 2021, 1, 1, 310, 5, 147);
 8: execute_matricula_insert(22, 2, 20, 2, 2021, 1, 10, 63, 5, 147);
 9: execute_matricula_insert(175, 2, 19, 1, 2020, 1, 7, 131, 5, 147);
10: execute_matricula_insert(222, 2, 32, 4, 2020, 1, 8, 228, 5, 147);

=== SUMMARY ===
Total matriculas records: 18785
Unique institutions: 15
Unique careers: 321
Unique genders: 2
Unique age ranges: 6
Unique entrance methods: 11
Comuna ID (constant): 147

✅ All matriculas data successfully processed and ready for database insertion!
